In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
# Read the data
df = pd.read_csv('init_preprocessed_data_without_index.csv')
len(df)

C:\Users\jaswi\AppData\Local\Temp\ipykernel_45292\3040050841.py:2: DtypeWarning: Columns (37,47,48,49,50,51,52,53,54,55,56,57,58,59,60,76,77) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('init_preprocessed_data_without_index.csv')


3252599

In [ ]:
df.columns

In [ ]:
df.head()

Create state fields

In [3]:
df.PWSTATE2.unique()

array([ 0,  1, 21, 39, 12, 47, 13, 28, 22, 86, 26, 17,  6, 55, 48, 27, 51,
       84, 45,  4, 34, 41, 37, 20,  8, 42,  5,  2, 40, 16, 54, 18, 29, 30,
       83, 19, 36, 49, 53, 15, 33, 35, 32, 31, 38,  9, 81, 11, 24, 23, 56,
       82, 85, 88, 25, 10, 46, 44, 50, 72], dtype=int64)

In [4]:
num_nostate = (df.PWSTATE2 == 0).values.sum()
original_len = len(df)
print(f"{num_nostate} ({num_nostate/original_len:.2%}) of the rows have no state")

1803778 (55.46%) of the rows have no state


In [5]:
# Remove rows with no state
df = df[df.PWSTATE2 != 0]
assert(len(df) == original_len - num_nostate)
print(f"Removed {num_nostate} rows with no state. {len(df)} rows remain.")

Removed 1803778 rows with no state. 1448821 rows remain.


In [3]:
with(open('ipums_fields/stateField.json')) as f:
    state_mapping = json.load(f)

In [ ]:
# Make sure each degree field is in the mapping
stateKeys = sorted([int(k) for k in state_mapping.keys()])
# Note that stateKeys doesn't include foreign countries
assert(sorted([val for val in df.PWSTATE2.unique().tolist() if val <= stateKeys[-1]]) == stateKeys)

In [ ]:
df["worksOutsideUS"] = (df.PWSTATE2 > stateKeys[-1])
cols_created = 1
for (key, value) in state_mapping.items():
    # Skip N/A column (may want to fill this with NaN later)
    if value == 'N/A':
        continue

    stateName = value.replace(' ', '_')
    df[f"worksIn_{stateName}"] = (df.PWSTATE2 == int(key))
    cols_created += 1

print(f"Created {cols_created} columns")

In [ ]:
df.drop(columns=['PWSTATE2', 'PWCOUNTY', 'PWTYPE'], inplace=True)

Create degree fields

In [ ]:
with(open('ipums_fields/degField.json')) as f:
    deg_mapping = json.load(f)

In [ ]:
# Make sure each degree field is in the mapping
assert(sorted(df.DEGFIELD.unique().tolist()) == sorted([int(k) for k in deg_mapping.keys()]))

In [ ]:
cols_created = 0
for (key, value) in deg_mapping.items():
    # Skip N/A column (may want to fill this with NaN later)
    if value == 'N/A':
        continue

    degName = value.replace(' ', '_').replace(',', '')
    df[f"hasDegree_{degName}"] = (df.DEGFIELD == int(key))
    cols_created += 1

print(f"Created {cols_created} columns")

In [ ]:
df.drop(columns=['DEGFIELD', 'DEGFIELDD'], inplace=True)

Create occupation fields

In [ ]:
with(open('ipums_fields/occupation2010.json')) as f:
    occ_mapping = json.load(f)

In [ ]:
# Make sure each degree field is in the mapping
assert(sorted(df.OCC2010.unique().tolist()) == sorted([int(k) for k in occ_mapping.keys()]))

In [ ]:
cols_created = 0
for (key, value) in occ_mapping.items():
    # Skip N/A column (may want to fill this with NaN later)
    if value == 'N/A':
        continue

    occName = value.replace(' ', '_').replace(',', '')
    df[f"occupation_{occName}"] = (df.OCC2010 == int(key))
    cols_created += 1

print(f"Created {cols_created} columns")

In [ ]:
df.drop(columns=['OCC2010'], inplace=True)

In [ ]:
df.reset_index().to_csv('large_field_preprocessed_data.csv', index=False)